In [25]:
import pandas as pd
import paramiko
import random
import numpy as np
import os
import glob
import datetime

In [2]:
host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "juba-jl1" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [3]:
files_folder="/mnt/drv5/jian/Big_Lots/Analysis/2020Q1_FB_mobile_test/files_from_SP/"
local_folder="./files_from_SP_20200408/" #MS team
try:
    os.stat(local_folder)
except:
    os.mkdir(local_folder)


list_remote_base_name=sftp.listdir(files_folder)
list_remote_base_name

['biglotsvisitor_1201_0315.csv',
 'combinedgrocery.csv',
 'biglotsvicinity_1201_0315.csv',
 'combineddicountstore6.csv']

In [4]:
for remote_file_path_bn in list_remote_base_name:
    remote_path=files_folder+remote_file_path_bn
    local_path=local_folder+remote_file_path_bn
    sftp.get(remote_path,local_path)
sftp.close()
transport.close()

In [6]:
list_all_4_segs=glob.glob(local_folder+"*.csv")
list_all_4_segs

['./files_from_SP_20200408/biglotsvisitor_1201_0315.csv',
 './files_from_SP_20200408/combinedgrocery.csv',
 './files_from_SP_20200408/biglotsvicinity_1201_0315.csv',
 './files_from_SP_20200408/combineddicountstore6.csv']

In [35]:
write_folder="./sliced_ids_by_segments/"

try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [36]:
dict_rename_seg={"visitor":"seg_1_BL_visitor",
                 "vacinity":"seg_2_BL_vacinity",
                 "combineddicountstore6":"seg_3_nonfurn_visitor",
                 "combinedgrocery":"seg_4_grocery_visitor"}

df_dict_control_pctg={"seg_1_BL_visitor":0.1,
                      "seg_2_BL_vacinity":0.02,
                      "seg_3_nonfurn_visitor":0.03,
                      "seg_4_grocery_visitor":0.06}

df_dict_phase1_pctg={"seg_1_BL_visitor":0.8,
                      "seg_2_BL_vacinity":0.1,
                      "seg_3_nonfurn_visitor":0.35,
                      "seg_4_grocery_visitor":0.2}

random.seed(0)

for file in list_all_4_segs:
    df=pd.read_csv(file,dtype=str)
    
    list_df_cols=df.columns.tolist()
    
    if 'group' in list_df_cols:
        seg_name_sp=df['group'].unique()[0]
        seg_name=dict_rename_seg[seg_name_sp]
    else:
        seg_name_sp=os.path.basename(file).split(".")[0]
        seg_name=dict_rename_seg[seg_name_sp]
        
    print(seg_name,df.shape)
    
    df=df.reset_index()[['index','ad_id']]
    len_df_total=len(df)
    list_C=random.sample(range(len_df_total),int(len_df_total*df_dict_control_pctg[seg_name]))

    df_control=df[df['index'].isin(list_C)]
    df_test=df[~df['index'].isin(list_C)]
    
    del df_test['index']
    df_test=df_test.reset_index()
    del df_test['index']
    df_test=df_test.reset_index()
    
    len_df_test=len(df_test)
    list_phase_1=random.sample(range(len_df_test),int(len_df_total*df_dict_phase1_pctg[seg_name])) # the pctg as the toal

    df_Test_phase_1=df_test[df_test['index'].isin(list_phase_1)]
    df_Remaining_phase_2=df_test[~df_test['index'].isin(list_phase_1)]
    
    print(df_control.shape,df_Test_phase_1.shape,df_Remaining_phase_2.shape)
    print(df_control['ad_id'].nunique(),df_Test_phase_1['ad_id'].nunique(),df_Remaining_phase_2['ad_id'].nunique())
    
    df_control.to_csv(write_folder+"Control_"+seg_name+".csv",index=False)
    df_Test_phase_1.to_csv(write_folder+"Test_Phase1_"+seg_name+".csv",index=False)
    df_Remaining_phase_2.to_csv(write_folder+"Remaing_Phase2"+seg_name+".csv",index=False)
    
    
    

seg_1_BL_visitor (2261850, 2)
(226185, 2) (1809480, 2) (226185, 2)
226185 1809480 226185
seg_4_grocery_visitor (3581153, 1)
(214869, 2) (716230, 2) (2650054, 2)
214869 716230 2650054
seg_2_BL_vacinity (9397562, 2)
(187951, 2) (939756, 2) (8269855, 2)
187951 939756 8269855
seg_3_nonfurn_visitor (6952687, 7)
(208580, 2) (2433440, 2) (4310667, 2)
208580 2433440 4310667


In [39]:
1809480/2261850

0.8

In [40]:
list_all_seg_sliced_files=glob.glob(write_folder+"*.csv")
list_all_seg_sliced_files.sort()
list_all_seg_sliced_files

['./sliced_ids_by_segments/Control_seg_1_BL_visitor.csv',
 './sliced_ids_by_segments/Control_seg_2_BL_vacinity.csv',
 './sliced_ids_by_segments/Control_seg_3_nonfurn_visitor.csv',
 './sliced_ids_by_segments/Control_seg_4_grocery_visitor.csv',
 './sliced_ids_by_segments/Remaing_Phase2seg_1_BL_visitor.csv',
 './sliced_ids_by_segments/Remaing_Phase2seg_2_BL_vacinity.csv',
 './sliced_ids_by_segments/Remaing_Phase2seg_3_nonfurn_visitor.csv',
 './sliced_ids_by_segments/Remaing_Phase2seg_4_grocery_visitor.csv',
 './sliced_ids_by_segments/Test_Phase1_seg_1_BL_visitor.csv',
 './sliced_ids_by_segments/Test_Phase1_seg_2_BL_vacinity.csv',
 './sliced_ids_by_segments/Test_Phase1_seg_3_nonfurn_visitor.csv',
 './sliced_ids_by_segments/Test_Phase1_seg_4_grocery_visitor.csv']

In [41]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
BigLotsAccount_id="act_271491453638620"

# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBAHsJtDc00ZBtZAW2ye9CRdKsy9ivLlzvgIMsEkZAyZA3mRpuGBF8IvZAR9obyYZCdzfw455nzMEW76A5ZBZBS6Fft2nQsUSs8sZCcGDZBZCbVeiCq0LMvekWjnnDZCuglkNihTAZAWIWTpvzaZBeZCKYywrPl0ZAsoWul36cyQZDZD"



def creatfbaudience(AudienceName,AudienceDescription,df,ACCESS_TOKEN,accountid,):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v6.0')
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    audience = CustomAudience(fbname)
    users = list(df['ad_id'])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(CustomAudience.Schema.mobile_advertiser_id, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)

In [42]:
for file in list_all_seg_sliced_files:
    audience_name = os.path.basename(file).split(".")[0]
    description=os.path.basename(file).split(".")[0]+"_"+str(datetime.datetime.now().date())
    df = pd.read_csv(file,dtype=str)
    creatfbaudience(AudienceName=audience_name,
                    AudienceDescription=description,
                    df=df,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id)

2020-04-08 20:46:24.102672 Control_seg_1_BL_visitor uploaded 226185
2020-04-08 20:46:46.485166 Control_seg_2_BL_vacinity uploaded 187951
2020-04-08 20:47:12.338491 Control_seg_3_nonfurn_visitor uploaded 208580
2020-04-08 20:47:34.566788 Control_seg_4_grocery_visitor uploaded 214869
2020-04-08 20:47:51.714313 Remaing_Phase2seg_1_BL_visitor uploaded 226185
2020-04-08 20:57:59.063859 Remaing_Phase2seg_2_BL_vacinity uploaded 8269855
2020-04-08 21:03:37.002140 Remaing_Phase2seg_3_nonfurn_visitor uploaded 4310667
2020-04-08 21:06:55.851836 Remaing_Phase2seg_4_grocery_visitor uploaded 2650054
2020-04-08 21:09:12.648100 Test_Phase1_seg_1_BL_visitor uploaded 1809480
2020-04-08 21:10:15.273687 Test_Phase1_seg_2_BL_vacinity uploaded 939756
2020-04-08 21:13:47.937036 Test_Phase1_seg_3_nonfurn_visitor uploaded 2433440
2020-04-08 21:14:49.077204 Test_Phase1_seg_4_grocery_visitor uploaded 716230
